In [47]:
# Do all imports and installs here - Done
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
import pandas as pd
import re
import configparser
import os

In [48]:
# Parse configurations - Done
config = configparser.ConfigParser()
config.read('etl.cfg')

['etl.cfg']

#### Path 
input_data_source = '.'

output_processed_data = './storage'

#

i94immi_data_source = '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'

worldtempe_data_source = '../../data2/GlobalLandTemperaturesByCity.csv'

citydemo_data_source = './us-cities-demographics.csv'

airport_data_source = './airport-codes_csv.csv'

#

i94_immi_splited_dir = './storage/.sas7bdat'

world_tempe_splited_dir = './storage/.csv'

In [114]:
# Create spark session
spark = SparkSession.builder\
            .config("spark.jars.repositories", "https://repos.spark-packages.org/")\
            .config("spark.jars.packages", "saurfang:spark-sas7bdat:2.0.0-s_2.11")\
            .enableHiveSupport()\
            .getOrCreate()

In [ ]:
# df = spark.read.options(header='True',inferSchema='True',delimiter=',').csv(worldtempe_data_source)
# df = spark.read.options(header='True',inferSchema='True',delimiter=';').csv(citydemo_data_source)
# df = spark.read.options(header='True',inferSchema='True',delimiter=',').csv(airport_data_source)
# df = spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')

In [49]:
# For production only
# worldtempe_dataset = '../../data2/GlobalLandTemperaturesByCity.csv'

worldtempe_dataset = 'GlobalLandTemperaturesByCity_part9.csv'
worldtempe_df = pd.read_csv(worldtempe_dataset,sep=",")
worldtempe_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 7 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   dt                             100000 non-null  object 
 1   AverageTemperature             96552 non-null   float64
 2   AverageTemperatureUncertainty  96552 non-null   float64
 3   City                           100000 non-null  object 
 4   Country                        100000 non-null  object 
 5   Latitude                       100000 non-null  object 
 6   Longitude                      100000 non-null  object 
dtypes: float64(2), object(5)
memory usage: 5.3+ MB


In [50]:
worldtempe_df['dt'] = pd.to_datetime(worldtempe_df['dt'])

In [51]:
worldtempe_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 7 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   dt                             100000 non-null  datetime64[ns]
 1   AverageTemperature             96552 non-null   float64       
 2   AverageTemperatureUncertainty  96552 non-null   float64       
 3   City                           100000 non-null  object        
 4   Country                        100000 non-null  object        
 5   Latitude                       100000 non-null  object        
 6   Longitude                      100000 non-null  object        
dtypes: datetime64[ns](1), float64(2), object(4)
memory usage: 5.3+ MB


In [52]:
worldtempe_df['year'] = worldtempe_df['dt'].dt.year

2013

In [57]:
# Range of years
worldtempe_df['year'].sort_values(ascending=False)

91916    2013
38604    2013
86685    2013
86686    2013
86687    2013
         ... 
63714    1743
63713    1743
3182     1743
3181     1743
55342    1743
Name: year, Length: 100000, dtype: int64

In [58]:
worldtempe_df['year'].nunique()

271

In [59]:
worldtempe_df['month'] = worldtempe_df['dt'].dt.month

In [60]:
worldtempe_df.isnull().values.any()

True

In [61]:
worldtempe_df.isnull().sum()

dt                                  0
AverageTemperature               3448
AverageTemperatureUncertainty    3448
City                                0
Country                             0
Latitude                            0
Longitude                           0
year                                0
month                               0
dtype: int64

In [65]:
worldtempe_df[worldtempe_df['AverageTemperature'].isnull()]

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude,year,month
0,1748-09-01,NaN,NaN,Belgorod,Russia,50.63N,36.76E,1748,9
1,1748-10-01,NaN,NaN,Belgorod,Russia,50.63N,36.76E,1748,10
2,1748-11-01,NaN,NaN,Belgorod,Russia,50.63N,36.76E,1748,11
3,1748-12-01,NaN,NaN,Belgorod,Russia,50.63N,36.76E,1748,12
4,1749-01-01,NaN,NaN,Belgorod,Russia,50.63N,36.76E,1749,1
...,...,...,...,...,...,...,...,...,...
99957,1812-09-01,NaN,NaN,Bhilwara,India,24.92N,74.48E,1812,9
99958,1812-10-01,NaN,NaN,Bhilwara,India,24.92N,74.48E,1812,10
99959,1812-11-01,NaN,NaN,Bhilwara,India,24.92N,74.48E,1812,11
99960,1812-12-01,NaN,NaN,Bhilwara,India,24.92N,74.48E,1812,12


In [63]:
def nan_percentage_calc(df):
    nan_demographics_df = pd.DataFrame(data=df.isnull().sum(), columns=['NaN'])
    nan_demographics_df.drop(nan_demographics_df[nan_demographics_df['NaN'] == 0].index, inplace = True)
    nan_demographics_df['% of NaN'] = (nan_demographics_df['NaN']/df.count())*100
    return nan_demographics_df

In [64]:
nan_percentage_calc(worldtempe_df)

,NaN,% of NaN
AverageTemperature,3448,3.571133
AverageTemperatureUncertainty,3448,3.571133


In [66]:
worldtempe_df.dropna(axis=0, inplace=True)

In [67]:
nan_percentage_calc(worldtempe_df)

,NaN,% of NaN
dt,NaN,NaN
AverageTemperature,NaN,NaN
AverageTemperatureUncertainty,NaN,NaN
City,NaN,NaN
Country,NaN,NaN
Latitude,NaN,NaN
Longitude,NaN,NaN
year,NaN,NaN
month,NaN,NaN


In [68]:
worldtempe_df.isna().sum()

dt                               0
AverageTemperature               0
AverageTemperatureUncertainty    0
City                             0
Country                          0
Latitude                         0
Longitude                        0
year                             0
month                            0
dtype: int64

In [69]:
worldtempe_df['City'].nunique()

39

In [70]:
worldtempe_df['Country'].nunique()

23

In [71]:
worldtempe_df = worldtempe_df[worldtempe_df['Country'] == 'United States']

In [72]:
# Dropping 3 columns

worldtempe_df = worldtempe_df.drop(['AverageTemperatureUncertainty', 'Latitude', 'Longitude'], axis=1)
worldtempe_df.columns

Index(['dt', 'AverageTemperature', 'City', 'Country', 'year', 'month'], dtype='object')

In [73]:
def convert_column_names(df):
    cols = df.columns
    column_name_changed = []

    for col in cols:
        new_column = col.lstrip().rstrip().lower().replace (" ", "_").replace ("-", "_") #strip beginning spaces, makes lowercase, add underscpre
        column_name_changed.append(new_column)

    df.columns = column_name_changed

In [74]:
convert_column_names(worldtempe_df)
worldtempe_df.columns

Index(['dt', 'averagetemperature', 'city', 'country', 'year', 'month'], dtype='object')

In [75]:
worldtempe_df.head()

,dt,averagetemperature,city,country,year,month
9033,1828-01-01,-1.977,Bellevue,United States,1828,1
9034,1828-02-01,0.720,Bellevue,United States,1828,2
9035,1828-03-01,4.114,Bellevue,United States,1828,3
9037,1828-05-01,10.139,Bellevue,United States,1828,5
9038,1828-06-01,14.195,Bellevue,United States,1828,6


In [76]:
worldtempe_df.tail()

,dt,averagetemperature,city,country,year,month
55337,2013-05-01,17.434,Berkeley,United States,2013,5
55338,2013-06-01,19.759,Berkeley,United States,2013,6
55339,2013-07-01,20.657,Berkeley,United States,2013,7
55340,2013-08-01,19.731,Berkeley,United States,2013,8
55341,2013-09-01,20.471,Berkeley,United States,2013,9


In [78]:
worldtempe_df.value_counts()

dt          averagetemperature  city      country        year  month
1828-01-01  -1.977              Bellevue  United States  1828  1        1
1956-11-01   12.420             Berkeley  United States  1956  11       1
1955-08-01   18.872             Berkeley  United States  1955  8        1
1955-09-01   11.868             Bellevue  United States  1955  9        1
             18.615             Berkeley  United States  1955  9        1
                                                                       ..
1898-06-01   12.996             Bellevue  United States  1898  6        1
             18.739             Berkeley  United States  1898  6        1
1898-07-01   15.054             Bellevue  United States  1898  7        1
             19.403             Berkeley  United States  1898  7        1
2013-09-01   20.471             Berkeley  United States  2013  9        1
Length: 4159, dtype: int64

In [79]:
# Saving the clean df to a new csv
worldtempe_df.to_csv('worldtempe_df_clean.csv', index=False)